In [ ]:
# Import libraries
import requests
import pandas as pd
from datetime import datetime

In [ ]:
# Authentication Setup

api_key = "c965de4dda51fe59aa6c522dcb519ed794e0082a3abe70d73ebd491c463da433"
headers = {
    "Accept": "application/json; application/vnd.esios-api-v1+json",
    "Content-Type": "application/json",
    "x-api-key": api_key
}

In [ ]:
# Making API Calls

url = "https://api.esios.ree.es/indicators/1001"

response = requests.get(url, headers = headers)

# Check for successful response
if response.status_code == 200:
  # Access and process the JSON data
  data = response.json()
  # Process the data further (e.g., print specific fields, store in a DataFrame)
else:
  print(f"Error: API request failed with status code {response.status_code}")